In [96]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import simfin as sf
from simfin.names import *

In [97]:
# Set the SimFin data-directory
sf.set_data_dir("C:\\dev\\trading\\tradesystem1\\data\\simfin")

In [98]:
#  Set your API Key
sf_api_key = "<your API key>"
sf.set_api_key(sf_api_key)

In [99]:
#  Configure SimFin hub with cache refresh params
variant = 'latest' #  latest or ttm
hub = sf.StockHub(market='us',
                  refresh_days=30,
                  refresh_days_shareprices=1)


In [100]:
#  Load quartely income in line line!!!
income_df = hub.load_income(variant='quarterly')  # annual or quarterly
income_df = income_df.reset_index()
for col in income_df.columns:
    print(col)
income_df['INCOME_GROWTH'] = income_df[OPERATING_INCOME].pct_change(periods=1)
#  Average income growth in the last 4 quarters
income_df['INCOME_GROWTH_MA4'] = income_df['INCOME_GROWTH'].rolling(4).mean()
#  Gross income growth last 4 quarters
income_mask = income_df['INCOME_GROWTH_MA4'] > 0.20

#  Calculate past 3 months
today = datetime.today()
today_date_str = today.strftime("%Y-%m-%d")
past_date = today - timedelta(days=90)
past_date_str = past_date.strftime("%Y-%m-%d")

#  Get last 3 months of reporting
income_mask &= ((income_df[REPORT_DATE] >= past_date_str) & (income_df[REPORT_DATE] <= today_date_str))

#  Format results
income_results = income_df[income_mask]
income_results = income_results.groupby(['Ticker']).mean()
income_results.reset_index(inplace=True)
income_results = income_results[['Ticker', 'INCOME_GROWTH_MA4']]
print(income_results)

Dataset "us-income-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Ticker
Report Date
SimFinId
Currency
Fiscal Year
Fiscal Period
Publish Date
Restated Date
Shares (Basic)
Shares (Diluted)
Revenue
Cost of Revenue
Gross Profit
Operating Expenses
Selling, General & Administrative
Research & Development
Depreciation & Amortization
Operating Income (Loss)
Non-Operating Income (Loss)
Interest Expense, Net
Pretax Income (Loss), Adj.
Abnormal Gains (Losses)
Pretax Income (Loss)
Income Tax (Expense) Benefit, Net
Income (Loss) from Continuing Operations
Net Extraordinary Gains (Losses)
Net Income
Net Income (Common)
   Ticker  INCOME_GROWTH_MA4
0     ADI           0.202279
1      AI           0.298943
2    AMBA           1.925286
3   AMSWA           0.422999
4    ASPU           0.305638
..    ...                ...
59   VALU           0.332183
60  VLGEA           0.737297
61    VMW           1.364544
62   VSCO           6.001084
63   WEWA           0.264045

[64 rows x 2 columns]

In [101]:
#  Get financial ratios
financial_ratios_df = hub.load_derived(variant='quarterly')  # annual or quarterly
financial_ratios_df = financial_ratios_df.reset_index()
for col in financial_ratios_df.columns:
    print(col)


#  Build the mask
#  Company’s current assets to its current liabilities.
financial_mask = (financial_ratios_df[CURRENT_RATIO] > 1.5)
# The debt ratio is defined as the ratio of total debt to total assets
financial_mask &= (financial_ratios_df[DEBT_RATIO] < 0.2)
financial_mask &= (financial_ratios_df[NET_PROFIT_MARGIN] > 0.15)
#  Get last 3 months of reporting
financial_mask &= ((financial_ratios_df[REPORT_DATE] >= past_date_str) & (financial_ratios_df[REPORT_DATE] <= today_date_str))

financial_selections_df = financial_ratios_df[financial_mask]
financial_results = financial_selections_df[['Ticker']]
print(financial_results)

Dataset "us-derived-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Ticker
Report Date
SimFinId
Currency
Fiscal Year
Fiscal Period
Publish Date
Restated Date
EBITDA
Total Debt
Free Cash Flow
Gross Profit Margin
Operating Margin
Net Profit Margin
Return on Equity
Return on Assets
Free Cash Flow to Net Income
Current Ratio
Liabilities to Equity Ratio
Debt Ratio
Earnings Per Share, Basic
Earnings Per Share, Diluted
Sales Per Share
Equity Per Share
Free Cash Flow Per Share
Dividends Per Share
Piotroski F-Score
Return On Invested Capital
Cash Return On Invested Capital
Dividend Payout Ratio
Net Debt / EBITDA
Net Debt / EBIT
      Ticker
10527    BKE
17860   CODA
18806   CPRT
28445   FAST
34697    HEI
44827   LOVE
49643    MOV
50851     MU
68159    SLP
68624   SMTC
71434   SWBI
78023   VALU
78305   VEEV
84041   ZDGE
84222     ZM


In [104]:
#  Price Ratio Screening
price_ratio_df = hub.load_derived_shareprices(variant='daily')  # daily or latest
price_ratio_df = price_ratio_df.reset_index()
for col in price_ratio_df.columns:
    print(col)

#  Build the mask
# The price-to-earnings ratio (P/E ratio) is the  current share price relative to its earnings per share (EPS)
price_ratio_mask = (price_ratio_df[PE_QUARTERLY] < 25)
#  The price-to-cash flow (P/CF) ratio compares a company's market value to its operating cash flow or
#  its stock price per share to operating cash flow per share
price_ratio_mask &= (price_ratio_df[PRICE_FCF_QUARTERLY] < 20)
#  Get last 3 months of reporting
price_ratio_mask &= ((price_ratio_df['Date'] >= past_date_str) & (price_ratio_df['Date'] <= today_date_str))

price_selections_df = price_ratio_df[price_ratio_mask]
price_results = price_selections_df.groupby(['Ticker']).mean()
price_results.reset_index(inplace=True)
price_results = financial_selections_df[['Ticker']]
print(price_results)

Ticker
Date
SimFinId
Market-Cap
Price to Earnings Ratio (quarterly)
Price to Earnings Ratio (ttm)
Price to Sales Ratio (quarterly)
Price to Sales Ratio (ttm)
Price to Book Value
Price to Free Cash Flow (quarterly)
Price to Free Cash Flow (ttm)
Enterprise Value
EV/EBITDA
EV/Sales
EV/FCF
Book to Market Value
Operating Income/EV
Altman Z Score
      Ticker
10527    BKE
17860   CODA
18806   CPRT
28445   FAST
34697    HEI
44827   LOVE
49643    MOV
50851     MU
68159    SLP
68624   SMTC
71434   SWBI
78023   VALU
78305   VEEV
84041   ZDGE
84222     ZM
